In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
import requests

c:\Users\saumi\anaconda3\envs\langchain-llama\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between a given base currency and a target currency
    """
    url = f"https://v6.exchangerate-api.com/v6/a2fa77b554a408039062c8f3/pair/{base_currency}/{target_currency}"

    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float,InjectedToolArg]) -> float:
    """
    given a currency conversion rate this calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate

In [3]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1756339201,
 'time_last_update_utc': 'Thu, 28 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1756425601,
 'time_next_update_utc': 'Fri, 29 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.7256}

In [4]:
convert.invoke({'base_currency_value':10, 'conversion_rate':87.7256})

877.256

In [5]:
# tool binding
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [7]:
messages = [HumanMessage('What is the conversion factor between USd and INR, and based on that can you convert 10 usd to inr')]

In [8]:
messages

[HumanMessage(content='What is the conversion factor between USd and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={})]

In [9]:
ai_message = llm_with_tools.invoke(messages)

In [17]:
messages.append(ai_message)

In [18]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'target_currency': 'INR', 'base_currency': 'USD'},
  'id': '9d2fe4b3-5a85-4abb-8d57-64e4a7683624',
  'type': 'tool_call'}]

In [19]:
import json
for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message_1 = get_conversion_factor.invoke(tool_call)
        print(tool_message_1)
        #fetch this conversion rate
        conversion_rate = json.loads(tool_message_1.content)['conversion_rate']
        #append this tool message to the messages list
        messages.append(tool_message_1)
    #execute the 2nd tool using the conversion rate from tool 1
    if tool_call['name'] == 'convert':
        # fetch the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message_2 = convert.invoke(tool_call)
        messages.append(tool_message_2)

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756339201, "time_last_update_utc": "Thu, 28 Aug 2025 00:00:01 +0000", "time_next_update_unix": 1756425601, "time_next_update_utc": "Fri, 29 Aug 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 87.7256}' name='get_conversion_factor' tool_call_id='9d2fe4b3-5a85-4abb-8d57-64e4a7683624'


In [20]:
messages

[HumanMessage(content='What is the conversion factor between USd and INR, and based on that can you convert 10 usd to inr', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756339201, "time_last_update_utc": "Thu, 28 Aug 2025 00:00:01 +0000", "time_next_update_unix": 1756425601, "time_next_update_utc": "Fri, 29 Aug 2025 00:00:01 +0000", "base_code": "USD", "target_code": "INR", "conversion_rate": 87.7256}', name='get_conversion_factor', tool_call_id='9d2fe4b3-5a85-4abb-8d57-64e4a7683624'),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1756339201, "time_last_update_utc": "Thu, 28 Aug 2025 00:00:01 +0000", "time_next_update_unix": 1756425601, "time_next_update_utc": "Fri, 

In [22]:
llm_with_tools.invoke(messages).content

''